In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
import statsmodels.api as sm


elec  = pd.read_csv("US_County_Level_Presidential_Results_12-16.csv")
county  = pd.read_csv("county_facts.csv")

In [2]:
data=elec[elec['county_name']!='Alaska'].merge(county,left_on='FIPS',right_on='fips',how='left')
data_ak=elec[elec['county_name']=='Alaska'].drop_duplicates(['votes_dem_2016','votes_gop_2016'])
data_ak['FIPS']=2000
data_ak=data_ak.merge(county,left_on='FIPS',right_on='fips',how='left')
data=pd.concat((data,data_ak),axis=0).sort_values('fips')
data['target_2016']=(data['votes_dem_2016']>data['votes_gop_2016'])*1
data['target_2012']=(data['votes_dem_2012']>data['votes_gop_2012'])*1



In [3]:
print(len(data))
print(data.columns)
print(data.isnull().sum())

3113
Index(['Unnamed: 0', 'combined_fips', 'votes_dem_2016', 'votes_gop_2016',
       'total_votes_2016', 'per_dem_2016', 'per_gop_2016', 'diff_2016',
       'per_point_diff_2016', 'state_abbr', 'county_name', 'FIPS',
       'total_votes_2012', 'votes_dem_2012', 'votes_gop_2012', 'county_fips',
       'state_fips', 'per_dem_2012', 'per_gop_2012', 'diff_2012',
       'per_point_diff_2012', 'fips', 'area_name', 'state_abbreviation',
       'PST045214', 'PST040210', 'PST120214', 'POP010210', 'AGE135214',
       'AGE295214', 'AGE775214', 'SEX255214', 'RHI125214', 'RHI225214',
       'RHI325214', 'RHI425214', 'RHI525214', 'RHI625214', 'RHI725214',
       'RHI825214', 'POP715213', 'POP645213', 'POP815213', 'EDU635213',
       'EDU685213', 'VET605213', 'LFE305213', 'HSG010214', 'HSG445213',
       'HSG096213', 'HSG495213', 'HSD410213', 'HSD310213', 'INC910213',
       'INC110213', 'PVY020213', 'BZA010213', 'BZA110213', 'BZA115213',
       'NES010213', 'SBO001207', 'SBO315207', 'SBO115207', 'S

In [4]:
df = data[['AGE295214', 'AGE775214', 'SEX255214', 'RHI125214', 'RHI225214',
       'RHI325214', 'RHI425214', 'RHI525214', 'RHI625214', 'RHI725214',
       'RHI825214', 'POP715213', 'POP645213', 'POP815213', 'EDU635213',
       'EDU685213', 'VET605213', 'LFE305213', 'HSG010214', 'HSG445213',
       'HSG096213', 'HSG495213', 'HSD410213', 'HSD310213', 'INC910213',
       'INC110213', 'PVY020213', 'BZA010213', 'BZA110213', 'BZA115213',
       'NES010213', 'SBO001207', 'SBO315207', 'SBO115207', 'SBO215207',
       'SBO515207', 'SBO415207', 'SBO015207', 'MAN450207', 'WTN220207',
       'RTN130207', 'RTN131207', 'AFN120207', 'BPS030214', 'LND110210',
       'POP060210', 'per_gop_2016']]

In [5]:
corr = df.corr(method = 'pearson')
cor2y = np.abs(corr.loc['per_gop_2016'])
cor22y = cor2y.sort_values()

cor22y

LND110210       0.016211
BZA115213       0.017635
AGE295214       0.034754
LFE305213       0.053984
EDU635213       0.090323
SBO515207       0.097759
RHI325214       0.113527
SBO115207       0.134685
RHI525214       0.141385
PVY020213       0.142331
SEX255214       0.169669
HSD310213       0.170743
RHI725214       0.188371
POP715213       0.198195
RHI625214       0.200045
INC110213       0.208196
RTN131207       0.234911
INC910213       0.236907
WTN220207       0.251530
POP060210       0.262629
MAN450207       0.267236
BPS030214       0.286972
SBO415207       0.306420
AFN120207       0.306532
NES010213       0.317739
AGE775214       0.323267
POP815213       0.326551
SBO001207       0.329608
BZA010213       0.349001
RTN130207       0.349794
BZA110213       0.351493
SBO015207       0.357109
HSD410213       0.363317
HSG010214       0.365450
VET605213       0.371663
POP645213       0.395248
SBO215207       0.404285
SBO315207       0.419201
RHI225214       0.425049
RHI425214       0.446764


In [6]:
variables = cor22y[cor22y > 0.3]
variables = variables.index
variables

Index(['SBO415207', 'AFN120207', 'NES010213', 'AGE775214', 'POP815213',
       'SBO001207', 'BZA010213', 'RTN130207', 'BZA110213', 'SBO015207',
       'HSD410213', 'HSG010214', 'VET605213', 'POP645213', 'SBO215207',
       'SBO315207', 'RHI225214', 'RHI425214', 'HSG445213', 'HSG495213',
       'EDU685213', 'RHI125214', 'RHI825214', 'HSG096213', 'per_gop_2016'],
      dtype='object')

In [7]:
df = data[['SBO415207', 'AFN120207', 'NES010213', 'AGE775214', 'POP815213',
       'SBO001207', 'BZA010213', 'RTN130207', 'BZA110213', 'SBO015207',
       'HSD410213', 'HSG010214', 'VET605213', 'POP645213', 'SBO215207',
       'SBO315207', 'RHI225214', 'RHI425214', 'HSG445213', 'HSG495213',
       'EDU685213', 'RHI125214', 'RHI825214', 'HSG096213', 'target_2016']]


In [8]:
X = data[['SBO415207', 'AFN120207', 'NES010213', 'AGE775214', 'POP815213',
       'SBO001207', 'BZA010213', 'RTN130207', 'BZA110213', 'SBO015207',
       'HSD410213', 'HSG010214', 'VET605213', 'POP645213', 'SBO215207',
       'SBO315207', 'RHI225214', 'RHI425214', 'HSG445213', 'HSG495213',
       'EDU685213', 'RHI125214', 'RHI825214', 'HSG096213']]

X['const'] = 1

y = data['per_gop_2016']

feature_columns = ['SBO415207', 'AFN120207', 'NES010213', 'AGE775214', 'POP815213',
       'SBO001207', 'BZA010213', 'RTN130207', 'BZA110213', 'SBO015207',
       'HSD410213', 'HSG010214', 'VET605213', 'POP645213', 'SBO215207',
       'SBO315207', 'RHI225214', 'RHI425214', 'HSG445213', 'HSG495213',
       'EDU685213', 'RHI125214', 'RHI825214', 'HSG096213']

C:\Users\sean\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:

#변수 선택법
def processSubset(x,y, feature_set):
    model = sm.OLS(y,x[list(feature_set)])  #modeling
    regr = model.fit()  #모델학습
    AIC = regr.aic    #모델의 AIC
    return {"model" : regr , "AIC" : AIC}
 
print(processSubset(X,y,feature_set=feature_columns))
 
#모든 조합을 다 조합해서 좋은 모델을 반환시키는 알고리즘
import time
import itertools
 
def getBest(x,y,k):
    tic = time.time()  #시작시간
    results = []       #결과저장공간
    for combo in itertools.combinations(x.columns.difference(['const']),k): 
        combo=(list(combo)+['const'])
        #각 변수조합을 고려한 경우의 수
        results.append(processSubset(x,y,feature_set=combo))#모델링된 것들을 저장
    models=pd.DataFrame(results) #데이터 프레임으로 변환
    #가장 낮은 AIC를 가지는 모델 선택 및 저장
    bestModel = models.loc[models['AIC'].argmin()] #index
    toc = time.time() #종료시간
    print("Processed",models.shape[0],"models on",k,"predictors in",(toc-tic),
          "seconds.")
    return  bestModel
 
#print(getBest(x=trainX,y=trainY,k=2))
 
#변수 선택에 따른 학습시간과 저장 K 반복
 
 
models = pd.DataFrame(columns=["AIC","model"])
tic = time.time()
for i in range(1,4):
    models.loc[i] = getBest(X,y,i)
 
#toc = time.time()
#print("Total elapsed time : ",(toc-tic),"seconds")
 
#print(models)
 
#전진 선택법(Step=1)
 
def forward(x,y,predictors):
    remainingPredictors = [p for p in x.columns.difference(['const'])
                           if p not in predictors]
    tic=time.time()
    results=[]
    for p in remainingPredictors:
        results.append(processSubset(x=x,y=y,feature_set=predictors+[p]+['const']))
    #데이터프레임으로 변환
    models = pd.DataFrame(results)
 
    #AIC가 가장 낮은 것을 선택
    bestModel = models.loc[models['AIC'].argmin()] #index
    toc = time.time()
    print("Processed ", models.shape[0],"models on", len(predictors)+1,
          "predictors in",(toc-tic))
    print("Selected predictors:",bestModel['model'].model.exog_names,
          'AIC : ',bestModel[0])
    return bestModel
""" 
#전진선택법 모델
def forward_model(x,y):
    fModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    #미리 정의된 데이터 변수
    predictors = []
    #변수1~10개 : 0~9 -> 1~10
    for i in range(1,len(x.columns.difference(['const']))+1):
        forwardResult= forward(x,y,predictors)
        if i > 1:
            if forwardResult['AIC'] > fmodelBefore:
                break
        fModels.loc[i] = forwardResult
        predictors = fModels.loc[i]["model"].model.exog_names
        fmodelBefore = fModels.loc[i]["AIC"]
        predictors = [k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapesed time : ", (toc - tic), "seconds.")
 
    return (fModels['model'][len(fModels['model'])])
 
forwordBestModel=forward_model(X,y)
 
print(forwordBestModel.summary())
"""

 
 
#후진제거법
 
def backward(x,y,predictors):
    tic = time.time()
    results=[]
    #데이터 변수들이 미리정의된 predictors 조합확인
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(x,y,list(combo)+['const']))
    models = pd.DataFrame(results)
    #가장 낮은 AIC를 가진 모델을 선택
    bestModel = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print("Processed",models.shape[0],"models on",len(predictors)-1,
          "predictors in",(toc - tic))
    print("Selected predictors :",bestModel['model'].model.exog_names,
          ' AIC:',bestModel[0])
    return bestModel
"""
def backword_model(x,y):
    BModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    #미리 정의된 데이터 변수
    predictors = x.columns.difference(['const'])
    BmodelBefore = processSubset(x,y,predictors)['AIC']
    while(len(predictors)>1):
        backwardResult=backward(X,y,predictors)
        if backwardResult['AIC'] > BmodelBefore:
            break
        BModels.loc[len(predictors)-1] = backwardResult
        predictors = BModels.loc[len(predictors)-1]["model"].model.exog_names
        BmodelBefore = backwardResult["AIC"]
        predictors = [ k for k in predictors if k != 'const']
 
    toc = time.time()
    print("Total elapsed time :",(toc - tic), "seconds.")
    return (BModels["model"].dropna().iloc[0])
 
backwardBestModel = backword_model(X,y)
"""
def Stepwise_model(x,y):
    stepModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    predictors = []
    SmodelBefore = processSubset(x,y,predictors+['const'])['AIC']
    
    for i in range(1, len(x.columns.difference(['const']))+1):
        forwardResult = forward(x,y,predictors)
        print("forward")
        stepModels.loc[i] = forwardResult
        predictors = stepModels.loc[i]["model"].model.exog_names
        predictors = [k for k in predictors if k != 'const']
        backwordResult = backward(x,y,predictors)
        if backwordResult['AIC'] < forwardResult['AIC']:
            stepModels.loc[i] = backwordResult
            predictors=stepModels.loc[i]["model"].model.exog_names
            smodelBefore=stepModels.loc[i]["AIC"]
            predictors=[k for k in predictors if k != 'const']
            print('backward')
        if stepModels.loc[i]["AIC"] > SmodelBefore:
            break
        else:
            smodelBefore = stepModels.loc[i]["AIC"]
    toc=time.time()
    print("Total elapsed time : ", (toc - tic), "seconds")
    return (stepModels['model'][len(stepModels['model'])])
 
stepwiseBestModel = Stepwise_model(X,y)


{'model': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002224F835548>, 'AIC': -5832.90848773861}
Processed 24 models on 1 predictors in 0.03124380111694336 seconds.
Processed 276 models on 2 predictors in 0.59688401222229 seconds.
Processed 2024 models on 3 predictors in 5.6608052253723145 seconds.
Processed  24 models on 1 predictors in 0.04686617851257324
Selected predictors: ['HSG096213', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000222746EC9C8>
forward
Processed 1 models on 0 predictors in 0.0
Selected predictors : ['const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000222736A82C8>
Processed  23 models on 2 predictors in 0.04686450958251953
Selected predictors: ['HSG096213', 'RHI125214', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002227A7554C8>
forward
Processed 2 models on 1 predictors in 0.0
Selected predictors

Processed  9 models on 16 predictors in 0.06267786026000977
Selected predictors: ['HSG096213', 'RHI125214', 'HSG495213', 'SBO415207', 'EDU685213', 'SBO215207', 'AGE775214', 'POP815213', 'POP645213', 'SBO315207', 'HSG445213', 'SBO015207', 'RHI425214', 'RHI225214', 'AFN120207', 'VET605213', 'const'] AIC :  <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000222746D9648>
forward
Processed 16 models on 15 predictors in 0.10102415084838867
Selected predictors : ['HSG096213', 'RHI125214', 'HSG495213', 'SBO415207', 'EDU685213', 'SBO215207', 'AGE775214', 'POP815213', 'POP645213', 'SBO315207', 'HSG445213', 'SBO015207', 'RHI425214', 'RHI225214', 'AFN120207', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000222746E6148>
Processed  8 models on 17 predictors in 0.031243085861206055
Selected predictors: ['HSG096213', 'RHI125214', 'HSG495213', 'SBO415207', 'EDU685213', 'SBO215207', 'AGE775214', 'POP815213', 'POP645213', 'SBO31520

In [10]:
stepwiseBestModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           per_gop_2016   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     357.2
Date:                Wed, 14 Oct 2020   Prob (F-statistic):               0.00
Time:                        19:20:29   Log-Likelihood:                 3164.9
No. Observations:                3113   AIC:                            -6290.
Df Residuals:                    3093   BIC:                            -6169.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
HSG096213     -0.0067      0.000    -19.394      0.000      -0.007      -0.006
RHI125214      0.0044      0.000     18.147      0.000       0.004       0.005
HSG495213  -5.242e-07   3.44e-08    -15.252      0.000   -5.92e-07   -4.57e-07
SBO415207     -0.0036      0.000     -9.832      0.000      -0.004      -0.003
EDU685213     -0.0027      0.000     -8.907      0.000      -0.003      -0.002
SBO215207      0.0082      0.001      5.709      0.000       0.005       0.011
AGE775214     -0.0035      0.000     -7.876      0.000      -0.004      -0.003
POP815213     -0.0043      0.000    -13.156      0.000      -0.005      -0.004
POP645213      0.0074      0.001     10.434      0.000       0.006       0.009
SBO315207     -0.0010      0.000     -3.103      0.002      -0.002      -0.000
HSG445213     -0.0015      0.000     -4.551      0.000      -0.002      -0.001
SBO015207     -0.0005      0.000     -3.484      0.001      -0.001      -0.000
RHI425214     -0.0033      0.002     -2.007      0.045      -0.007   -7.51e-05
RHI225214     -0.0007      0.000     -2.609      0.009      -0.001      -0.000
NES010213   7.886e-07   3.22e-07      2.448      0.014    1.57e-07    1.42e-06
HSD410213  -1.536e-06   2.93e-07     -5.233      0.000   -2.11e-06    -9.6e-07
HSG010214   9.952e-07   2.48e-07      4.021      0.000     5.1e-07    1.48e-06
BZA010213   5.641e-06   1.63e-06      3.457      0.001    2.44e-06    8.84e-06
RTN130207  -4.872e-09   2.57e-09     -1.895      0.058   -9.91e-09    1.69e-10
const          0.6579      0.030     22.200      0.000       0.600       0.716
==============================================================================
Omnibus:                       64.215   Durbin-Watson:                   1.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               74.504
Skew:                          -0.299   Prob(JB):                     6.63e-17
Kurtosis:                       3.464   Cond. No.                     8.18e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.18e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
'''
#변수 선택법
def processSubset(x,y, feature_set):
    model = LogisticRegressionCV(cv=5, random_state=0)  #modeling
    regr = model.fit(x[list(feature_set)],y) #모델학습
    Score = regr.score(x,y)    #모델의 스코어
    return {"model" : regr , "Score" : Score}
    
 
print(processSubset(X,y,feature_set=feature_columns))
 
#모든 조합을 다 조합해서 좋은 모델을 반환시키는 알고리즘
import time
import itertools
 
def getBest(x,y,k):
    tic = time.time()  #시작시간
    results = []       #결과저장공간
    for combo in itertools.combinations(x.columns.difference(['const']),k): 
        combo=(list(combo))
        #각 변수조합을 고려한 경우의 수
        results.append(processSubset(x,y,feature_set=combo))#모델링된 것들을 저장
    models=pd.DataFrame(results) #데이터 프레임으로 변환
    #가장 높은 Score를 가지는 모델 선택 및 저장
    bestModel = models.loc[models['Score'].argmax()] #index
    toc = time.time() #종료시간
    print("Processed",models.shape[0],"models on",k,"predictors in",(toc-tic),
          "seconds.")
    return  bestModel
 
#print(getBest(x=trainX,y=trainY,k=2))
 
#변수 선택에 따른 학습시간과 저장 K 반복
 
 
models = pd.DataFrame(columns=["Score","model"])
tic = time.time()
for i in range(1,4):
    models.loc[i] = getBest(X,y,i)
 
#toc = time.time()
#print("Total elapsed time : ",(toc-tic),"seconds")
 
#print(models)
 
#전진 선택법(Step=1)
 
def forward(x,y,predictors):
    remainingPredictors = [p for p in x.columns.difference(['const'])
                           if p not in predictors]
    tic=time.time()
    results=[]
    for p in remainingPredictors:
        results.append(processSubset(x=x,y=y,feature_set=predictors+[p]+['const']))
    #데이터프레임으로 변환
    models = pd.DataFrame(results)
 
    #가장 높은 Score를 가지는 모델 선택 및 저장
    bestModel = models.loc[models['Score'].argmax()] #index
    toc = time.time()
    print("Processed ", models.shape[0],"models on", len(predictors)+1,
          "predictors in",(toc-tic))
    print("Selected predictors:",bestModel['model'].model.exog_names,
          'Score : ',bestModel[0])
    return bestModel
""" 
#전진선택법 모델
def forward_model(x,y):
    fModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    #미리 정의된 데이터 변수
    predictors = []
    #변수1~10개 : 0~9 -> 1~10
    for i in range(1,len(x.columns.difference(['const']))+1):
        forwardResult= forward(x,y,predictors)
        if i > 1:
            if forwardResult['AIC'] > fmodelBefore:
                break
        fModels.loc[i] = forwardResult
        predictors = fModels.loc[i]["model"].model.exog_names
        fmodelBefore = fModels.loc[i]["AIC"]
        predictors = [k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapesed time : ", (toc - tic), "seconds.")
 
    return (fModels['model'][len(fModels['model'])])
 
forwordBestModel=forward_model(X,y)
 
print(forwordBestModel.summary())
"""

 
 
#후진제거법
 
def backward(x,y,predictors):
    tic = time.time()
    results=[]
    #데이터 변수들이 미리정의된 predictors 조합확인
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(x,y,list(combo)+['const']))
    models = pd.DataFrame(results)
    #가장 높은 Score를 가지는 모델 선택 및 저장
    bestModel = models.loc[models['Score'].argmax()] #index
    toc = time.time()
    print("Processed",models.shape[0],"models on",len(predictors)-1,
          "predictors in",(toc - tic))
    print("Selected predictors :",bestModel['model'].model.exog_names,
          ' Score:',bestModel[0])
    return bestModel
"""
def backword_model(x,y):
    BModels = pd.DataFrame(columns=["AIC","model"])
    tic = time.time()
    #미리 정의된 데이터 변수
    predictors = x.columns.difference(['const'])
    BmodelBefore = processSubset(x,y,predictors)['AIC']
    while(len(predictors)>1):
        backwardResult=backward(X,y,predictors)
        if backwardResult['AIC'] > BmodelBefore:
            break
        BModels.loc[len(predictors)-1] = backwardResult
        predictors = BModels.loc[len(predictors)-1]["model"].model.exog_names
        BmodelBefore = backwardResult["AIC"]
        predictors = [ k for k in predictors if k != 'const']
 
    toc = time.time()
    print("Total elapsed time :",(toc - tic), "seconds.")
    return (BModels["model"].dropna().iloc[0])
 
backwardBestModel = backword_model(X,y)
"""
def Stepwise_model(x,y):
    stepModels = pd.DataFrame(columns=["Score","model"])
    tic = time.time()
    predictors = []
    SmodelBefore = processSubset(x,y,predictors+['const'])['Score']
    #변수 1~10개 : 0~9 -> 1~10
    for i in range(1, len(x.columns.difference(['const']))+1):
        forwardResult = forward(x,y,predictors)
        print("forward")
        stepModels.loc[i] = forwardResult
        predictors = stepModels.loc[i]["model"].model.exog_names
        predictors = [k for k in predictors if k != 'const']
        backwordResult = backward(x,y,predictors)
        if backwordResult['Score'] > forwardResult['Score']:
            stepModels.loc[i] = backwordResult
            predictors=stepModels.loc[i]["model"].model.exog_names
            smodelBefore=stepModels.loc[i]["Score"]
            predictors=[k for k in predictors if k != 'const']
            print('backward')
        if stepModels.loc[i]["Score"] < SmodelBefore:
            break
        else:
            smodelBefore = stepModels.loc[i]["Score"]
    toc=time.time()
    print("Total elapsed time : ", (toc - tic), "seconds")
    return (stepModels['model'][len(stepModels['model'])])
 
stepwiseBestModel = Stepwise_model(X,y)
'''

'\n#변수 선택법\ndef processSubset(x,y, feature_set):\n    model = LogisticRegressionCV(cv=5, random_state=0)  #modeling\n    regr = model.fit(x[list(feature_set)],y) #모델학습\n    Score = regr.score(x,y)    #모델의 스코어\n    return {"model" : regr , "Score" : Score}\n    \n \nprint(processSubset(X,y,feature_set=feature_columns))\n \n#모든 조합을 다 조합해서 좋은 모델을 반환시키는 알고리즘\nimport time\nimport itertools\n \ndef getBest(x,y,k):\n    tic = time.time()  #시작시간\n    results = []       #결과저장공간\n    for combo in itertools.combinations(x.columns.difference([\'const\']),k): \n        combo=(list(combo))\n        #각 변수조합을 고려한 경우의 수\n        results.append(processSubset(x,y,feature_set=combo))#모델링된 것들을 저장\n    models=pd.DataFrame(results) #데이터 프레임으로 변환\n    #가장 높은 Score를 가지는 모델 선택 및 저장\n    bestModel = models.loc[models[\'Score\'].argmax()] #index\n    toc = time.time() #종료시간\n    print("Processed",models.shape[0],"models on",k,"predictors in",(toc-tic),\n          "seconds.")\n    return  bestModel\n \n#print(getB

In [12]:
#stepwiseBestModel.summary()

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


Final_X = data[['HSG096213', 'RHI125214', 'HSG495213', 'SBO415207', 'EDU685213', 'SBO215207', 'AGE775214', 
                'POP815213', 'POP645213', 'SBO315207', 'HSG445213', 'SBO015207', 'RHI425214', 'RHI225214', 
                'NES010213', 'HSD410213', 'HSG010214', 'BZA010213', 'RTN130207']]
y = data['target_2016']

x_train, x_test, y_train, y_test = model_selection.train_test_split(Final_X, y, test_size=0.2)

clf1 = LogisticRegression(C=1, max_iter=1000).fit(x_train, y_train)


In [14]:
y_pred2 = clf1.predict(x_test)

print("Logistic model's Accuracy score is :",accuracy_score(y_test, y_pred2))
print("Logistic model's Presision score is :",precision_score(y_test, y_pred2))
print("Logistic model's Recall score is :",recall_score(y_test, y_pred2))
print("Logistic model's F1 score is :",f1_score(y_test, y_pred2))

Logistic model's Accuracy score is : 0.9020866773675762
Logistic model's Presision score is : 0.8181818181818182
Logistic model's Recall score is : 0.46875
Logistic model's F1 score is : 0.5960264900662251


In [15]:
Final_X = data[['HSG096213', 'RHI125214', 'HSG495213', 'SBO415207', 'EDU685213', 'SBO215207', 'AGE775214', 
                'POP815213', 'POP645213', 'SBO315207', 'HSG445213', 'SBO015207', 'RHI425214', 'RHI225214', 
                'NES010213', 'HSD410213', 'HSG010214', 'BZA010213', 'RTN130207']]
y = data['target_2016']

clf2 = LogisticRegressionCV(cv=5, random_state=0).fit(Final_X, y)


C:\Users\sean\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\sean\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

In [16]:
y_pred = clf2.predict(Final_X)


print("LogisticCV model's Accuracy score is :",accuracy_score(y, y_pred))
print("LogisticCV model's Presision score is :",precision_score(y, y_pred))
print("LogisticCV model's Recall score is :",recall_score(y, y_pred))
print("LogisticCV model's F1 score is :",f1_score(y, y_pred))

LogisticCV model's Accuracy score is : 0.8904593639575972
LogisticCV model's Presision score is : 0.8356164383561644
LogisticCV model's Recall score is : 0.375
LogisticCV model's F1 score is : 0.5176803394625177


In [17]:
data['target_2016'].value_counts()

0    2625
1     488
Name: target_2016, dtype: int64

In [18]:
data['target_2012'].value_counts()

0    2427
1     686
Name: target_2012, dtype: int64

In [25]:
data.shape

(3113, 77)